# [WIP] Fine Tuning GPT-3.5-Turbo with Function Calls

In this notebook, we walk through an example of fine-tuning gpt-3.5-turbo with function calls, through distilling GPT-4.

In [ ]:
# !pip install llama-index pypdf sentence-transformers ragas

In [1]:
import os
import openai

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

## Define GPT-4 Pydantic Program

Here, we define the GPT-4 powered function calling program that will generate structured outputs into a Pydantic object (an Album).

In [20]:
from llama_index.program import OpenAIPydanticProgram
from pydantic import BaseModel
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager
from typing import List


class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]


finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

llm = OpenAI(model="gpt-4", callback_manager=callback_manager)


prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = OpenAIPydanticProgram.from_defaults(
    output_cls=Album,
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=False,
)

## Log Inputs/Outputs

We define some sample movie names as inputs and log the outputs through the function calling program.

In [30]:
# NOTE: we need >= 10 movies to use OpenAI fine-tuning
movie_names = [
    "The Shining",
    "The Departed",
    "Titanic",
    "Goodfellas",
    "Pretty Woman",
    "Home Alone",
    "Caged Fury",
    "Edward Scissorhands",
    "Total Recall",
    "Ghost",
    "Tremors",
    "RoboCop",
    "Rocky V",
]

In [23]:
from tqdm.notebook import tqdm

for movie_name in tqdm(movie_names):
    output = program(movie_name=movie_name)
    print(output.json())

  0%|          | 0/13 [00:00<?, ?it/s]

{"name": "The Shining", "artist": "Various Artists", "songs": [{"title": "Main Title", "length_seconds": 180}, {"title": "Opening Credits", "length_seconds": 120}, {"title": "The Overlook Hotel", "length_seconds": 240}, {"title": "Redrum", "length_seconds": 150}, {"title": "Here's Johnny!", "length_seconds": 200}]}
{"name": "The Departed Soundtrack", "artist": "Various Artists", "songs": [{"title": "Gimme Shelter", "length_seconds": 272}, {"title": "Comfortably Numb", "length_seconds": 383}, {"title": "I'm Shipping Up to Boston", "length_seconds": 166}, {"title": "Sweet Dreams (Are Made of This)", "length_seconds": 216}, {"title": "I'm Shipping Up to Boston (Instrumental)", "length_seconds": 166}, {"title": "The Departed Tango", "length_seconds": 123}, {"title": "Thief's Theme", "length_seconds": 201}, {"title": "Well Well Well", "length_seconds": 126}, {"title": "Comfortably Numb (Live)", "length_seconds": 383}, {"title": "Sail On, Sailor", "length_seconds": 181}]}
{"name": "Titanic S

In [24]:
finetuning_handler.save_finetuning_events("mock_finetune_songs.jsonl")

Wrote 13 examples to mock_finetune_songs.jsonl


In [ ]:
!cat mock_finetune_songs.jsonl

## Fine-tune on the Dataset

We now define a fine-tuning engine and fine-tune on the mock dataset.

In [27]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "mock_finetune_songs.jsonl",
    # start_job_id="<start-job-id>"  # if you have an existing job, can specify id here
    validate_json=False,  # openai validate json code doesn't support function calling yet
)

In [28]:
finetune_engine.finetune()

In [33]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-uJ9kQ9pI0p0YNatBDxF3VITv at 0x172a5c9a0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-uJ9kQ9pI0p0YNatBDxF3VITv",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696463378,
  "finished_at": 1696463749,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:llamaindex::8660TXqx",
  "organization_id": "org-1ZDAvajC6v2ZtAP9hLEIsXRz",
  "result_files": [
    "file-Hbpw15BAwyf3e4HK5Z9g4IK2"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-MNh7snhv0triDIhsrErokSMY",
  "hyperparameters": {
    "n_epochs": 7
  },
  "trained_tokens": 22834,
  "error": null
}

## Try it Out! 

We obtain the fine-tuned LLM and use it with the Pydantic program.

In [34]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

In [35]:
ft_program = OpenAIPydanticProgram.from_defaults(
    output_cls=Album,
    prompt_template_str=prompt_template_str,
    llm=ft_llm,
    verbose=False,
)

In [37]:
ft_program(movie_name="Goodfellas")

Album(name='Goodfellas Soundtrack', artist='Various Artists', songs=[Song(title='Rags to Riches', length_seconds=180), Song(title='Gimme Shelter', length_seconds=270), Song(title='Layla', length_seconds=270), Song(title='Jump into the Fire', length_seconds=240), Song(title='Atlantis', length_seconds=180), Song(title='Beyond the Sea', length_seconds=180), Song(title='Sunshine of Your Love', length_seconds=240), Song(title='Mannish Boy', length_seconds=240), Song(title='Layla (Piano Exit)', length_seconds=120)])